In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import glob
import os

# 2023 Data

In [2]:
parquet_urls_23 = [
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-04.parquet'
]

#### Pull all links for 2023 data into a single DataFrame

In [3]:
all_files = glob.glob('parquet_urls_23')
dfs_23 = []
for url in parquet_urls_23:
    df = pd.read_parquet(url)
    dfs_23.append(df)

df23 = pd.concat(dfs_23, ignore_index=True)

#### Checking out what might be muddying up the data

In [4]:
#How many trips have total distance with negative or 0 distance?
df23_no_distance = df23[df23['trip_distance'] <= 0]
count_df23_no_distance = df23_no_distance.shape[0]
count_df23_no_distance

175722

In [5]:
#How many trips have total fares listed with negative or 0 value?
df23_negative_total = df23[df23['total_amount'] <= 0]
count_df23_negative_total = df23_negative_total.shape[0]
count_df23_negative_total

111873

In [6]:
#Which payment types are showing negative or 0 values?
#Most are 4 (Dispute), 2 (Cash), 3 (No charge)
#https://www.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf
df23_negative_total.payment_type.value_counts()

payment_type
4    64460
2    26093
3    20184
1      955
0      180
5        1
Name: count, dtype: int64

#### Add column for percent of total fare the taxi riders have tipped

In [7]:
df23['tip_percent'] = (df23['tip_amount'] / df23['fare_amount']) * 100

#### To clean the data, make df with only rides that have fares and distance over 0

In [8]:
df23_cleaned = df23[(df23['total_amount'] > 0) & (df23['trip_distance'] > 0)]

#### Make df that shows total ride count by Month and Year
> This includes all payment types

In [9]:
df23_count = df23_cleaned.groupby([df23_cleaned['tpep_pickup_datetime'].dt.year.rename('year'), df23_cleaned['tpep_pickup_datetime'].dt.month.rename('month')]).agg(count=('VendorID', 'size')).reset_index()
df23_count 

,year,month,count
0,2001,1,6
1,2002,12,3
2,2003,1,4
3,2008,12,10
4,2009,1,1
5,2014,11,1
6,2022,10,6
7,2022,12,25
8,2023,1,2998608
9,2023,2,2850852


> We can see that the data includes dates other than 2023. Unclear if it's an error, so removing all other dates

In [10]:
df23_count_clean = df23_count[df23_count['year'] == 2023]
df23_count_clean

,year,month,count
8,2023,1,2998608
9,2023,2,2850852
10,2023,3,3328989
11,2023,4,3221451
12,2023,5,46


#### Want to calculate how often people pay with card vs. cash
> <p> Filtering out other payment types that aren't card or cash </p>

In [11]:
df23_paymenttype = df23_cleaned[(df23_cleaned['payment_type'] == 1) | (df23_cleaned['payment_type'] == 2)]

In [12]:
pay_breakdown_23 = df23_paymenttype.payment_type.value_counts()
pay_breakdown_23

payment_type
1    9896483
2    2083509
Name: count, dtype: int64

>Make a dataframe out of the breakdown by cash and card

In [13]:
pay_breakdown_23_df = pd.DataFrame({'Payment Type': pay_breakdown_23.index, 'Count': pay_breakdown_23.values})
pay_breakdown_23_df['Payment Type'] = pay_breakdown_23_df['Payment Type'].replace({1: 'Credit Card', 2: 'Cash'})
pay_breakdown_23_df['Year'] = 2023
pay_breakdown_23_df

,Payment Type,Count,Year
0,Credit Card,9896483,2023
1,Cash,2083509,2023


#### Want to calculate median tip percentage by month 
> <p> But tip isn't documented when payment type is cash, so make a new df only for trips when payment type is credit card (1) to calculate median the riders are tipping when paying with card </p>

In [14]:
df23_credit = df23_cleaned[(df23_cleaned['payment_type'] == 1)]

#### Make df that shows median tip percentage by Month and Year when payment method is Credit Card

In [15]:
df23_tip = df23_credit.groupby([df23_credit['tpep_pickup_datetime'].dt.year.rename('year'), df23_credit['tpep_pickup_datetime'].dt.month.rename('month')]).agg(count=('VendorID', 'size'), median_tip=('tip_percent', 'median')).reset_index()
df23_tip

,year,month,count,median_tip
0,2001,1,3,19.498607
1,2008,12,4,25.360901
2,2014,11,1,26.374622
3,2022,10,3,20.205128
4,2022,12,20,31.198569
5,2023,1,2390453,26.134969
6,2023,2,2289885,26.062378
7,2023,3,2667597,25.925926
8,2023,4,2548478,25.882353
9,2023,5,39,24.878049


> Removing all dates besides 2023

In [16]:
df23_tip_clean = df23_tip[df23_tip['year'] == 2023]
df23_tip_clean

,year,month,count,median_tip
5,2023,1,2390453,26.134969
6,2023,2,2289885,26.062378
7,2023,3,2667597,25.925926
8,2023,4,2548478,25.882353
9,2023,5,39,24.878049


# 2022 Data

In [17]:
parquet_urls_22 = [
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-03.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-04.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-05.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-06.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-07.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-08.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-09.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-10.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-11.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-12.parquet'
]

#### Put all links for 2022 data into single df

In [18]:
all_files = glob.glob('parquet_urls_22')
dfs_22 = []
for url in parquet_urls_22:
    df = pd.read_parquet(url)
    dfs_22.append(df)

df22 = pd.concat(dfs_22, ignore_index=True)

#### Add column for percent of total fare the taxi riders have tipped

In [19]:
df22['tip_percent'] = (df22['tip_amount'] / df22['fare_amount']) * 100

#### To clean the data, make df with only rides that have fares and distance over 0

In [20]:
df22_cleaned = df22[(df22['total_amount'] > 0) & (df22['trip_distance'] > 0)]

### Total Ride Count by Month and Year
> This includes all payment types, and cleaned to only include 2022 data

In [21]:
df22_count = df22_cleaned.groupby([df22_cleaned['tpep_pickup_datetime'].dt.year.rename('year'), df22_cleaned['tpep_pickup_datetime'].dt.month.rename('month')]).agg(count=('VendorID', 'size')).reset_index()
df22_count_clean = df22_count[df22_count['year'] == 2022]
df22_count_clean

,year,month,count
8,2022,1,2423144
9,2022,2,2933259
10,2022,3,3569044
11,2022,4,3542145
12,2022,5,3524118
13,2022,6,3486393
14,2022,7,3110137
15,2022,8,3084151
16,2022,9,3113320
17,2022,10,3590655


### Credit Card vs. Cash Payments
> <p> Filtering out other payment types that aren't card or cash and making it a df </p>

In [22]:
df22_paymenttype = df22_cleaned[(df22_cleaned['payment_type'] == 1) | (df22_cleaned['payment_type'] == 2)]

In [23]:
pay_breakdown_22 = df22_paymenttype.payment_type.value_counts()
pay_breakdown_22_df = pd.DataFrame({'Payment Type': pay_breakdown_22.index, 'Count': pay_breakdown_22.values})
pay_breakdown_22_df['Payment Type'] = pay_breakdown_22_df['Payment Type'].replace({1: 'Credit Card', 2: 'Cash'})
pay_breakdown_22_df['Year'] = 2022
pay_breakdown_22_df

,Payment Type,Count,Year
0,Credit Card,29796585,2022
1,Cash,7569962,2022


### Median tip percentage by month
> only for trips where payment method is credit card (1)

In [24]:
df22_credit = df22_cleaned[(df22_cleaned['payment_type'] == 1)]
df22_tip = df22_credit.groupby([df22_credit['tpep_pickup_datetime'].dt.year.rename('year'), df22_credit['tpep_pickup_datetime'].dt.month.rename('month')]).agg(count=('VendorID', 'size'), median_tip=('tip_percent', 'median')).reset_index()
df22_tip_clean = df22_tip[df22_tip['year'] == 2022]
df22_tip_clean

,year,month,count,median_tip
6,2022,1,1858410,27.209302
7,2022,2,2278343,26.833333
8,2022,3,2762264,26.666667
9,2022,4,2710326,26.521739
10,2022,5,2694724,26.285714
11,2022,6,2668172,26.285714
12,2022,7,2342891,26.354839
13,2022,8,2328896,26.370370
14,2022,9,2389709,26.080000
15,2022,10,2777541,26.156250


# 2021 Data

In [25]:
parquet_urls_21 = [
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-02.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-03.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-04.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-05.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-06.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-07.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-08.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-09.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-10.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-11.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-12.parquet'
]

In [26]:
all_files = glob.glob('parquet_urls_21')
dfs_21 = []
for url in parquet_urls_21:
    df = pd.read_parquet(url)
    dfs_21.append(df)
df21 = pd.concat(dfs_21, ignore_index=True)

#### Add column for percent of total fare the taxi riders have tipped

In [27]:
df21['tip_percent'] = (df21['tip_amount'] / df21['fare_amount']) * 100

#### To clean the data, make df with only rides that have fares and distance over 0

In [28]:
df21_cleaned = df21[(df21['total_amount'] > 0) & (df21['trip_distance'] > 0)]

### Total Ride Count by Month and Year
> This includes all payment types, and cleaned to only include 2021 data

In [29]:
df21_count = df21_cleaned.groupby([df21_cleaned['tpep_pickup_datetime'].dt.year.rename('year'), df21_cleaned['tpep_pickup_datetime'].dt.month.rename('month')]).agg(count=('VendorID', 'size')).reset_index()
df21_count_clean = df21_count[df21_count['year'] == 2021]
df21_count_clean

,year,month,count
7,2021,1,1343633
8,2021,2,1346651
9,2021,3,1892358
10,2021,4,2130852
11,2021,5,2464340
12,2021,6,2788830
13,2021,7,2773242
14,2021,8,2739741
15,2021,9,2897170
16,2021,10,3408413


### Credit Card vs. Cash Payments
> <p> Filtering out other payment types that aren't card or cash and making it a df </p>

In [30]:
df21_paymenttype = df21_cleaned[(df21_cleaned['payment_type'] == 1) | (df21_cleaned['payment_type'] == 2)]

In [31]:
pay_breakdown_21 = df21_paymenttype.payment_type.value_counts()
pay_breakdown_21_df = pd.DataFrame({'Payment Type': pay_breakdown_21.index, 'Count': pay_breakdown_21.values})
pay_breakdown_21_df['Payment Type'] = pay_breakdown_21_df['Payment Type'].replace({1: 'Credit Card', 2: 'Cash'})
pay_breakdown_21_df['Year'] = 2021
pay_breakdown_21_df

,Payment Type,Count,Year
0,Credit Card,22245974,2021
1,Cash,6536472,2021


### Median tip percentage by month
> only for trips where payment method is credit card (1)

In [32]:
df21_credit = df21_cleaned[(df21_cleaned['payment_type'] == 1)]
df21_tip = df21_credit.groupby([df21_credit['tpep_pickup_datetime'].dt.year.rename('year'), df21_credit['tpep_pickup_datetime'].dt.month.rename('month')]).agg(count=('VendorID', 'size'), median_tip=('tip_percent', 'median')).reset_index()
df21_tip_clean = df21_tip[df21_tip['year'] == 2021]
df21_tip_clean

,year,month,count,median_tip
6,2021,1,921461,27.600000
7,2021,2,938264,27.333333
8,2021,3,1331336,27.333333
9,2021,4,1531027,27.166667
10,2021,5,1807630,26.947368
11,2021,6,2058432,26.677966
12,2021,7,2007970,26.842105
13,2021,8,1984739,26.793103
14,2021,9,2145458,26.500000
15,2021,10,2585226,26.500000


# 2020 Data

In [33]:
parquet_urls_20 = [
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-01.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-02.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-03.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-04.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-05.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-06.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-07.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-08.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-09.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-10.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-11.parquet',
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-12.parquet'
]

In [34]:
all_files = glob.glob('parquet_urls_20')
dfs_20 = []
for url in parquet_urls_20:
    df = pd.read_parquet(url)
    dfs_20.append(df)
df20 = pd.concat(dfs_20, ignore_index=True)

#### Add column for percent of total fare the taxi riders have tipped

In [35]:
df20['tip_percent'] = (df20['tip_amount'] / df20['fare_amount']) * 100

#### To clean the data, make df with only rides that have fares and distance over 0

In [36]:
df20_cleaned = df20[(df20['total_amount'] > 0) & (df20['trip_distance'] > 0)]

### Total Ride Count by Month and Year
> This includes all payment types, and cleaned to only include 2020 data

In [37]:
df20_count = df20_cleaned.groupby([df20_cleaned['tpep_pickup_datetime'].dt.year.rename('year'), df20_cleaned['tpep_pickup_datetime'].dt.month.rename('month')]).agg(count=('VendorID', 'size')).reset_index()
df20_count_clean = df20_count[df20_count['year'] == 2020]
df20_count_clean

,year,month,count
3,2020,1,6317110
4,2020,2,6220581
5,2020,3,2965620
6,2020,4,230531
7,2020,5,336835
8,2020,6,530115
9,2020,7,772188
10,2020,8,975017
11,2020,9,1312700
12,2020,10,1650166


### Credit Card vs. Cash Payments
> <p> Filtering out other payment types that aren't card or cash and making it a df </p>

In [38]:
df20_paymenttype = df20_cleaned[(df20_cleaned['payment_type'] == 1) | (df20_cleaned['payment_type'] == 2)]

In [39]:
pay_breakdown_20 = df20_paymenttype.payment_type.value_counts()
pay_breakdown_20_df = pd.DataFrame({'Payment Type': pay_breakdown_20.index, 'Count': pay_breakdown_20.values})
pay_breakdown_20_df['Payment Type'] = pay_breakdown_20_df['Payment Type'].replace({1: 'Credit Card', 2: 'Cash'})
pay_breakdown_20_df['Year'] = 2020
pay_breakdown_20_df

,Payment Type,Count,Year
0,Credit Card,17255529,2020
1,Cash,6050409,2020


### Median tip percentage by month
> only for trips where payment method is credit card (1)

In [40]:
df20_credit = df20_cleaned[(df20_cleaned['payment_type'] == 1)]
df20_tip = df20_credit.groupby([df20_credit['tpep_pickup_datetime'].dt.year.rename('year'), df20_credit['tpep_pickup_datetime'].dt.month.rename('month')]).agg(count=('VendorID', 'size'), median_tip=('tip_percent', 'median')).reset_index()
df20_tip_clean = df20_tip[df20_tip['year'] == 2020]
df20_tip_clean

,year,month,count,median_tip
3,2020,1,4653312,26.600000
4,2020,2,4669249,26.500000
5,2020,3,2191233,26.370370
6,2020,4,127412,27.647059
7,2020,5,168198,27.387960
8,2020,6,310764,26.981132
9,2020,7,472831,27.222222
10,2020,8,617008,27.222222
11,2020,9,879348,27.083333
12,2020,10,1151564,26.947368


# Create combined df and csv for each output
> To be able to combine with 2019 data that I had to analyze separately because it kept crashing the kernel

2020-2023 Ride Count

In [41]:
df_counts = pd.concat([df20_count_clean, df21_count_clean, df22_count_clean, df23_count_clean], ignore_index=True)

In [43]:
df_counts.to_csv('df_counts.csv', index = False)

2020-2022 Payment Method Breakdown
> Leaving off 2023 because it's not complete and I'll be visualizing total percent difference

In [44]:
df_pay_breakdowns = pd.concat([pay_breakdown_20_df, pay_breakdown_21_df, pay_breakdown_22_df], ignore_index=True)

In [45]:
df_pay_breakdowns.to_csv('df_pay_breakdowns.csv', index = False)

2020-2023 Median Tip Percentage

In [46]:
df_tips = pd.concat([df20_tip_clean, df21_tip_clean, df22_tip_clean, df23_tip_clean], ignore_index=True)

In [47]:
df_tips.to_csv('df_tips.csv', index = False)